## Application of ECOSTRESS Data: Exploring the imapcts of Urban Heat Islands

In this notebook, we will demonstrate a simple example of using ECOSTRESS data together with GIS layers for the Boulder Colorado area to investigate spatial variation in Land Surface Temperature (LST). We will explore temperature variation within the Boulder, CO area as well as explore the relationship of LST and tree cover using the Boulder CO Open Tree Data

### Step 1. Setup notebook

First, we need to import packages and set up some environment variables for the notebook.

In [ ]:
# Import Packages
import os, time, shutil
import warnings

import numpy as np
import cartopy.crs as ccrs
import rioxarray as rxr
import hvplot.xarray
import hvplot.pandas
import holoviews as hv
import matplotlib.pyplot as plt
import rasterio
import rasterio.plot
import geoviews as gv
import geopandas as gpd
import shapely

# Some cells may generate warnings that we can ignore. Comment below lines to see.
warnings.filterwarnings('ignore')

# Set some default plotting options.
size_opts = dict(frame_width=800, frame_height=600, fontscale=1.5)
map_opts = dict(
    geo=True, tiles='EsriImagery',
    rot=90,
    xlabel='Longitude', ylabel='Latitude')

#### Transfer project data from the cloud

We want to copy over data stored in CyVerse so that we can access files quickly for analysis. To do this, we will use Python "shutil" package to copy files from the CyVerse data store to a local, temporary location in the HYR-SENSE GitHub repository. The data we are copying is stored in a shared ESIIL / HYR-SENSE location. We can copy the files just from the "Agriculture" module which contains data for this exercise. However, if you want to work with other modules, they can be accessed here as well.

In [ ]:
### Setup notebook data and copy to scratch/working folder
# Identify the location of the HYR-SENSE "data store"
data_store_path = '/data-store/iplant/home/shared/esiil/HYR_SENSE/data/01-Urban-Heat-Island'
# Set a destination path (this is a 'local' and temporary path)
dest = os.path.join(os.path.expanduser("~"),"HYR-SENSE","data","Urban_heat")
if not os.path.exists(dest):
    os.mkdir(dest) # create the directory for the copied data, if needed
    
# Using 'shutil' package, copy all the files over
shutil.copytree(data_store_path, dest, dirs_exist_ok=True)

print(" ")
print(" ")
print("*** Copy complete! ***")
print(" ")

### Step 2. Load Data

We will now load the notebook data, including the GIS layers and the ECOSTRESS image

In [ ]:
### Import the City of Boulder boundary GIS layer

# Define a path to the boundary
boulder_path = os.path.join(os.path.expanduser("~"),"HYR-SENSE",
    # GIS directory
    'data', 'Urban_heat',
    # City of Boulder file
    'City_of_Boulder_City_Limits.zip'
)

# Read file and merge geometries
boulder_gdf = gpd.read_file(boulder_path).dissolve()

# Check that we have a GeoDataFrame
boulder_gdf

In [ ]:
### Load the tree inventory data
# Define a path to the open tree data
trees_path = os.path.join(os.path.expanduser("~"),"HYR-SENSE",
    # GIS directory
    'data', 'Urban_heat',
    # City of Boulder file
    'Tree_Inventory_Open_Data.zip'
)
# Read file and merge geometries
#trees_gdf = gpd.read_file(trees_path).dissolve()
trees_gdf = gpd.read_file(trees_path)
trees_gdf = trees_gdf[trees_gdf.geometry!=None]

# Clip to the ROI
trees_gdf = trees_gdf.sjoin(boulder_gdf)

# Check that we have a GeoDataFrame
trees_gdf

In [ ]:
### Load the example ECOSTRESS layer covering Boulder, CO

# Define the filename to use:  ECOSTRESS L2T LSTE
eco_file = 'ECOv002_L2T_LSTE_28527_009_13TDE_20230718T081442_0710_01_LST.tif'
# Define ECOSTRESS file path
data_dir = os.path.join(os.path.expanduser("~"),"HYR-SENSE","data","Urban_heat")

# Open the LSTE file using open_rasterio from the rioxarray library
eco_path = os.path.join(data_dir,eco_file)
eco_lst_ds = (
    rxr.open_rasterio(eco_path)
    # There is only 1 band, so we can squeeze and remove the band dimension.
    .squeeze('band', drop=True)
)
eco_lst_ds

### Step 3. Explore the GIS layers

ADD NOTES HERE

Let's start by looking at the GIS layer we loaded that delineates the city limits of Boulder, CO

In [ ]:
### Plot the GIS boundary for Boulder, Colorado
boulder_roi_plot = (boulder_gdf.hvplot(**map_opts,
                    line_color='orange', line_width=3,
                    fill_color='white', fill_alpha=.05) * 
 gpd.GeoDataFrame(geometry=boulder_gdf.envelope).hvplot(**map_opts,
        line_color='skyblue', fill_color=None)
).opts(**size_opts)
boulder_roi_plot

In [ ]:
### Plot the tree locations
trees_plot = trees_gdf.hvplot.points(
    **map_opts,
    size=15
).opts(**size_opts)
trees_plot

### Step 4. Prepare the ECOSTRESS data for analysis

Now let's load the ECOSTRESS image to show what data we will be working with in this notebook

In [ ]:
### Load the ECOSTRESS data
eco_lst_ds_plot = (eco_lst_ds.rio.reproject('EPSG:4326')
 .hvplot.image(x='x', y='y', **size_opts, 
               cmap='inferno', tiles='ESRI', 
               xlabel='Longitude', ylabel='Latitude', 
               title='ECOSTRESS LST (K)', 
               crs='EPSG:4326'))
eco_lst_ds_plot

Above you can see a strong temperature gradients fro west to east moving from higher altitude to lower altitudes around the Boulder, CO area.  You will also see the data is presented in degrees Kelvin.  We can easily change degrees Kelvin to Degrees Celsius, which we will work with in the next steps 

In [ ]:
### Convert degrees Kelvin to degrees Celsius
eco_lst_ds_degC = eco_lst_ds
eco_lst_ds_degC.values = eco_lst_ds_degC.values-273.15
eco_lst_ds_degC

In [ ]:
### Confirm we have data in degrees C
eco_lst_ds_degC_plot = (eco_lst_ds_degC.rio.reproject('EPSG:4326')
 .hvplot.image(x='x', y='y', **size_opts, 
               cmap='inferno', tiles='ESRI', 
               xlabel='Longitude', ylabel='Latitude', 
               title='ECOSTRESS LST (degC)', 
               crs='EPSG:4326'))
eco_lst_ds_degC_plot

In [ ]:
# Cleanup - remove original ECOSTRESS LST object in degrees Kelvin
del eco_lst_ds

Let's also overlay the GIS boundary on the ECOSTRESS data to ensure we have data that includes Boulder, CO

In [ ]:
### Load the ECOSTRESS data and overlay the GIS boundary
eco_plot = (
    eco_lst_ds_degC
    .rio.reproject('EPSG:4326')
    .hvplot.image(
        x='x', y='y', **size_opts, 
        cmap='inferno', tiles='ESRI', 
        xlabel='Longitude', ylabel='Latitude', 
        title='ECOSTRESS LST (degC)', 
        crs='EPSG:4326')
)
gis_boundary_plot = (
    boulder_gdf.hvplot(
        **map_opts,
        line_color='black', line_width=5,
        fill_color='white', fill_alpha=.05,
    )
    *
    gpd.GeoDataFrame(geometry=boulder_gdf.envelope).hvplot(
        **map_opts,
        line_color='skyblue', fill_color=None
    )
).opts(**size_opts)
combined_plot = eco_plot * gis_boundary_plot
combined_plot

We can also plot the data with the Boulder Open Tree data

In [ ]:
### Plot again with the tree locations
eco_plot = (
    eco_lst_ds_degC
    .rio.reproject('EPSG:4326')
    .hvplot.image(
        x='x', y='y', **size_opts, 
        cmap='inferno', tiles='ESRI', 
        xlabel='Longitude', ylabel='Latitude', 
        title='ECOSTRESS LST (degC)', 
        crs='EPSG:4326')
)
gis_boundary_plot = (
    boulder_gdf.hvplot(
        **map_opts,
        line_color='black', line_width=5,
        fill_color='white', fill_alpha=.05,
    )
    *
    gpd.GeoDataFrame(geometry=boulder_gdf.envelope).hvplot(
        **map_opts,
        line_color='skyblue', fill_color=None
    )
).opts(**size_opts)
trees_plot = trees_gdf.hvplot.points(
    **map_opts,
    size=15
).opts(**size_opts)
combined_plot = eco_plot * gis_boundary_plot * trees_plot
combined_plot

In the interactive plot above you can pan around the ECOSTRESS LST image as well as zoom into the Boulder Colorado area to view the LST variation across the city and see the location of trees provided by the Open Tree Inventory Data. What patterns can you see in the larger image moving from west to east across the image which reflects a change in elevation and increasing population density?  What patterns can you see as you zoom into Boulder and in relation to the location of dense tree cover?

### Step 5. Analyze the ECOSTRESS data

Next, we will dive deeper into the data.  First let's generate some statistics using the GIS layers and the ECOSTRESS image

#### Step 5a. Explore LST variation across the image using transects

In [ ]:
# Create a transect from west to east that intersects with the Boulder city ROI - we will use this to create an LST transect plot
coords = [[-106.0, 40.02], [-105.0, 40.02]]
lst_transect = shapely.LineString(coords)
lst_transect_utm = gpd.GeoSeries(lst_transect, crs=4326).to_crs(32613)
lst_transect_utm = lst_transect_utm.iloc[0]
distances = np.arange(0, lst_transect_utm.length, 250)
lst_transect_pnt = [lst_transect_utm.interpolate(distance) for distance in distances]
lst_transect_pnt = gpd.GeoSeries(lst_transect_pnt, crs=32613).to_crs('EPSG:4326')
# Convert to a geo data frame
lst_transect_pnt = gpd.GeoDataFrame(geometry=lst_transect_pnt)
lst_transect_pnt

In [ ]:
# Extract values of LST to each point along our previously generated transect of points
lst_transect_pnt['LST'] = lst_transect_pnt.geometry.apply(
    lambda geom: eco_lst_ds_degC.rio.reproject('EPSG:4326').sel(x=geom.x, y=geom.y, 
                                                                method="nearest").values)
# Insert the distance data from west to east in meters by 250m steps
lst_transect_pnt['dist'] = distances
lst_transect_pnt

In [ ]:
# Create a figure showing the transect and the profile of LST values along the transect
fig, ax = plt.subplots(figsize=(8,6))
np.clip(eco_lst_ds_degC.rio.reproject('EPSG:4326').
        squeeze(),0,30).plot.imshow(cmap='jet')
plt.title("ECOSTRESS LST Transect")
gpd.GeoSeries(lst_transect).plot(ax=ax, color='black', 
                                 linewidth=2)
boulder_gdf.plot(ax=ax, color='none', edgecolor='white');

# LST profile
fig, ax = plt.subplots(figsize=(8,6))
lst_transect_pnt.set_index('dist')['LST'].plot(ax=ax, 
                                               color='black',
                                               linewidth=2)
ax.set_xlabel('Distance (m)')
ax.set_ylabel('LST (degC)');

#### Step 5b. Crop the ECOSTRESS data to the Boulder CO 

In [ ]:
# Crop ECOSTRESS data to Region of Interest
# ---

boulder_lst_da = (eco_lst_ds_degC
                  .rio.reproject('EPSG:4326')
                  .rio.clip(boulder_gdf.
                            to_crs('EPSG:4326').geometry))

In [ ]:
### Plot the data cropped to the Boulder CO ROI
eco_plot = (
    boulder_lst_da
    .rio.reproject('EPSG:4326')
    .hvplot.image(
        x='x', y='y', **size_opts, 
        cmap='inferno', tiles='ESRI', 
        xlabel='Longitude', ylabel='Latitude', 
        title='ECOSTRESS LST (degC)', 
        crs='EPSG:4326')
)
gis_boundary_plot = (
    boulder_gdf.hvplot(
        **map_opts,
        line_color='black', line_width=5,
        fill_color='white', fill_alpha=.05,
    )
    *
    gpd.GeoDataFrame(geometry=boulder_gdf.envelope).hvplot(
        **map_opts,
        line_color='skyblue', fill_color=None
    )
).opts(**size_opts)
trees_plot = trees_gdf.hvplot.points(
    **map_opts,
    size=3
).opts(**size_opts)
combined_plot = eco_plot * gis_boundary_plot * trees_plot
combined_plot

#### Step 5c. Summarize the LST data for the tree cover areas

In [ ]:
# Find the LST nearest each tree location
# ---

trees_gdf['LST'] = trees_gdf.geometry.apply(
    lambda geom: boulder_lst_da.sel(x=geom.x, y=geom.y, method="nearest").values)
trees_gdf

Review the table above which contains the information on individual trees in Boulder CO, their genus/species, and other information, and a new column added to the table containing the ECOSTRESS LST values in degrees C for the closest pixel to the location of each tree

In [ ]:
# Plot LST distribution for trees vs. total LST distribution
# ---

tree_lst_plot = (trees_gdf.LST.hvplot.violin(frame_width=600, frame_height=400, fontscale=1.5,
        label='Tree Cover', violin_fill_color='forestgreen', box_fill_color='forestgreen')
    * boulder_lst_da.to_dataframe(name='LST').LST.hvplot.violin(
        label='Total', violin_fill_color='skyblue', box_fill_color='skyblue'))
tree_lst_plot

### What do you observe?

*Write your response here*